In [2]:
import numpy as np
import torch

In [39]:
class Zernike:
    def __init__(self, order=6, 
                 locxy=torch.tensor([], dtype=torch.float32, requires_grad=True), 
                 deltaxy=torch.tensor([], dtype=torch.float32, requires_grad=True)):
        self.order = order 
        self.locxy = locxy.clone().detach() # torch.tensor(locxy, dtype=torch.float32, requires_grad=True)
        self.deltaxy = deltaxy.clone().detach() #torch.tensor(deltaxy, dtype=torch.float32, requires_grad=True)
        
    def poly(self, powerx, powery, x, y):
        return (x**powerx) * (y**powery)

    def poly_dx(self, powerx, powery, x, y):
        if powerx < 1:
            return 0.0
        else:
            return powerx * self.poly(powerx-1, powery, x, y)

    def poly_dy(self, powerx, powery, x, y):
        if powery < 1:
            return 0.0
        else:
            return powery * self.poly(powerx, powery-1, x, y)
            
    def zernikePoly(self, i, x, y):
        switcher={
                1: self.poly(0, 0, x, y),
                2: self.poly(0, 1, x, y),
                3: self.poly(1, 0, x, y),
                4: 2*self.poly(1, 1, x, y),
                5: 1 + 2*self.poly(0, 2, x, y) + 2*self.poly(2, 0, x, y), # 1 + 2*x^2 + 2*y^2 
                6: self.poly(1, 1, x, y),
                7: -3*self.poly(3,0,x,y) + 3*self.poly(1,2,x,y), # Z(3,0) = -x3 + 3xy2
                8: -2*self.poly(0,1,x,y) + 3*self.poly(0,3,x,y) + 3*self.poly(2,1,x,y),	# Z(3,1) = -2x + 3x3 + 3xy2 ((0),(-2,0),(0,0,0),(3,0,3,0))
                9: -2*self.poly(1,0,x,y) + 3*self.poly(3,0,x,y) + 3*self.poly(1,2,x,y), #  Z(3,2) = -2y + 3y3 + 3x2y ((0),(0,-2),(0,0,0),(0,3,0,3))
                10:   self.poly(3,0,x,y) - 3*self.poly(1,2,x,y), #  Z(3,3) = y3 - 3x2y ((0),(0,0),(0,0,0),(0,-3,0,1))
                11: -4*self.poly(1,3,x,y) + 4*self.poly(3,1,x,y), #  Z(4,0) = -4x3y + 4xy3 ((0),(0,0),(0,0,0),(0,0,0,0),(0,-4,0,4,0))
                12: -6*self.poly(1,1,x,y) + 8*self.poly(1,3,x,y) + 8*self.poly(3,1,x,y), #  Z(4,1) = -6xy + 8x3y + 8xy3 ((0),(0,0),(0,-6,0),(0,0,0,0),(0,8,0,8,0))
                13: self.poly(0,0,x,y) - 6*self.poly(0,2,x,y) - 6*self.poly(2,0,x,y)+6*self.poly(0,4,x,y)+12*self.poly(2,2,x,y)+6*self.poly(4,0,x,y), #  Z(4,2) = 1 - 6x2 - 6y2 + 6x4 + 12x2y2 + 6y4 ((1),(0,0),(-6,0,-6),(0,0,0,0),(6,0,12,0,6))
                14: 3*self.poly(0,2,x,y) - 3*self.poly(2,0,x,y) - 4*self.poly(0,4,x,y)+4*self.poly(4,0,x,y), #  Z(4,3) = 3x2 - 3y2 - 4x4 + 4y4 ((0),(0,0),(3,0,-3),(0,0,0,0),(-4,0,0,0,4))
                15: self.poly(0,4,x,y) - 6*self.poly(2,2,x,y) + self.poly(4,0,x,y), #  Z(4,4) = x4 - 6x2y2 + y4 ((0),(0,0),(0,0,0),(0,0,0,0),(1,0,-6,0,1))
                16:   self.poly(0,5,x,y) - 10*self.poly(2,3,x,y) + 5*self.poly(4,1,x,y), #  Z(5,0) = x5 - 10x3y2 + 5xy4
                17:  4*self.poly(0,3,x,y) - 12*self.poly(2,1,x,y) - 5*self.poly(0,5,x,y)+10*self.poly(2,3,x,y)+15*self.poly(4,1,x,y), # Z(5,1) = 4x3 - 12xy2 - 5x5 + 10x3y2 + 15xy4
                18:  3*self.poly(0,1,x,y) - 12*self.poly(0,3,x,y) - 12*self.poly(2,1,x,y)+10*self.poly(0,5,x,y) + 20*self.poly(2,3,x,y) + 10*self.poly(4,1,x,y), # Z(5,2) = 3x - 12x3 - 12xy2 + 10x5 + 20x3y2 + 10xy4
                19:  3*self.poly(1,0,x,y) - 12*self.poly(3,0,x,y) - 12*self.poly(1,2,x,y)+10*self.poly(5,0,x,y)+20*self.poly(3,2,x,y)+10*self.poly(1,4,x,y), # Z(5,3) = 3y - 12y3 - 12x2y + 10y5 + 20x2y3 + 10x4y
                20: -4*self.poly(3,0,x,y) + 12*self.poly(1,2,x,y) + 5*self.poly(5,0,x,y)-10*self.poly(3,2,x,y)-15*self.poly(1,4,x,y), # Z(5,4) = -4y3 + 12x2y + 5y5 - 10x2y3 - 15x4y
                21:   self.poly(5,0,x,y)-10*self.poly(3,2,x,y)+5*self.poly(1,4,x,y), # Z(5,5) = y5 - 10x2y3 + 5x4y
             }
        zPoly = switcher.get(i, "Invalid Zernike Index")
        return zPoly #switcher.get(i,"Invalid Zernike index")

order = 2
z = Zernike(order, torch.tensor([1, 1], dtype=torch.float32), 
                   torch.tensor([2, 2], dtype=torch.float32))

#print(z.zernikePoly(21, torch.tensor([2,2], requires_grad=True, 3))

In [40]:
xten = torch.tensor([2], dtype=torch.float32, requires_grad=True)
yten = torch.tensor([3], dtype=torch.float32, requires_grad=True)
#for i in range(5):
zPoly = z.zernikePoly(5, xten, yten)
zPoly.backward()
print(zPoly)
print(xten.grad, yten.grad)

zPoly = z.zernikePoly(6, xten, yten)
zPoly.backward()
print(zPoly)
print(xten.grad, yten.grad)

tensor([27.], grad_fn=<AddBackward0>)
tensor([8.]) tensor([12.])
tensor([6.], grad_fn=<MulBackward0>)
tensor([11.]) tensor([14.])


In [38]:
for i in range(5):
    zPoly = z.zernikePoly(i+1, xten, yten)
    zPoly.backward()
    
    print(zPoly.item(), xten.grad, yten.grad)
    xten.grad.zero_()
    yten.grad.zero_()

1.0 tensor([0.]) tensor([0.])
3.0 tensor([0.]) tensor([1.])
2.0 tensor([1.]) tensor([0.])
12.0 tensor([6.]) tensor([4.])
27.0 tensor([8.]) tensor([12.])


In [23]:
x = torch.tensor([2], dtype=torch.float32, requires_grad=True)
y = torch.tensor([3], dtype=torch.float32, requires_grad=True)
z = (x*x) + (y*y)

In [24]:
z.backward()

dz/dx=4.00, dz/dy=6.00


In [40]:
xx = x.detach().numpy()
xx

array([2.], dtype=float32)

In [51]:
A = torch.randn((3,5))
b = torch.randn(3)

x = torch.linalg.solve(A@A.transpose(0,1), b)

In [54]:
print((A@A.transpose(0,1)@x, b))

(tensor([-1.2310, -0.4170,  1.0353]), tensor([-1.2310, -0.4170,  1.0353]))


In [1]:
def week(i, x, y):
    switcher={
        0:'Sunday',
        1:'Monday',
        2:'Tuesday',
        3:'Wednesday',
        4:'Thursday',
        5:'Friday',
        6:'Saturday'
         }
    return switcher.get(i,"Invalid day of week")
    
week(3, 1, 2)

'Wednesday'

In [2]:
from torchvision import models